In [1]:
import pandas as pd

/var/folders/7w/2j_g09_j491gg_pz45w_m11r0000gn/T/ipykernel_18449/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [12]:
path = "../../../Dataset/UNSW-NB15/UNSW_NB15_training-set.csv"
dataset = pd.read_csv(path)
dataset["attack_cat"].value_counts()
dataset.drop(columns=["id", "label"], inplace=True)
for column in dataset.columns:
    if column in ["proto", "service", "state"]:
        encode_text_dummy(dataset, column)
    elif column == "attack_cat":
        encode_text_index(dataset, column)
    else:
        encode_numeric_zscore(dataset, column)
x, y = to_xy(dataset, 'attack_cat')
print(y)

/var/folders/7w/2j_g09_j491gg_pz45w_m11r0000gn/T/ipykernel_18449/3842793451.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[dummy_name] = dummies[x]
/var/folders/7w/2j_g09_j491gg_pz45w_m11r0000gn/T/ipykernel_18449/3842793451.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[dummy_name] = dummies[x]
/var/folders/7w/2j_g09_j491gg_pz45w_m11r0000gn/T/ipykernel_18449/3842793451.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perf

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
print(y.shape)

(175341, 10)


In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
from sklearn import preprocessing


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):

    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]

    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = f"{name}-{tv}"
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    """ Return a maping dict """
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    label_dict = dict(zip(le.classes_, le.transform(le.classes_)))
    logging.info(label_dict)  # Output: {'apple': 0, 'banana': 1, 'orange': 2}
    return label_dict


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd

In [11]:
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)